# 第 6 单元：在 Panda-Gym 中使用机器人模拟 Advantage Actor Critic (A2C) 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail"/>

在本 notebook 中，你将学习如何将 A2C 与 [Panda-Gym](https://github.com/qgallouedec/panda-gym) 结合使用。你将**训练一个机械臂**（Franka Emika Panda 机器人）来执行一项任务：

- `Reach`：机器人必须将其末端执行器放置在目标位置。

完成之后，你将能够**在其他机器人任务中进行训练**。


### 🎮 环境：

- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

### 📚 强化学习库：

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

我们一直在努力改进我们的教程，因此**如果你在本 notebook 中发现任何问题**，请在 [GitHub 仓库上提出一个 issue](https://github.com/huggingface/deep-rl-class/issues)。

## 本 notebook 的目标 🏆

在本 notebook 的最后，你将能够：

- 使用环境库 **Panda-Gym**。
- 使用 A2C **训练机器人**。
- 理解为什么**我们需要对输入进行归一化**。
- 将你训练好的 agent 和代码**推送到 Hub**，并附上精彩的视频回放和评估分数 🔥。




## 本 notebook 来自深度强化学习课程
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

在这门免费课程中，你将：

- 📖 **从理论到实践**学习深度强化学习。
- 🧑‍💻 学习**使用著名的深度强化学习库**，如 Stable Baselines3, RL Baselines3 Zoo, CleanRL 和 Sample Factory 2.0。
- 🤖 在**独特的环境中训练 agent**

更多内容请查看 📚 课程大纲 👉 https://simoninithomas.github.io/deep-rl-course

别忘了**<a href="http://eepurl.com/ic5ZUD">注册课程</a>**（我们收集你的电子邮件是为了在每个单元发布时**向你发送链接，并为你提供有关挑战和更新的信息）。**


保持联系的最佳方式是加入我们的 Discord 服务器，与社区和我们进行交流 👉🏻 https://discord.gg/ydHrjt3WP5

## 先决条件 🏗️
在深入学习本 notebook 之前，你需要：

🔲 📚 [阅读第 6 单元](https://huggingface.co/deep-rl-course/unit6/introduction)学习 Actor-Critic 方法 🤗  

# 让我们开始训练我们的第一个机器人 🤖

要为[认证流程](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process)验证此实践环节，你需要将训练好的模型推送到 Hub，并获得以下结果：

- 在 `PandaReachDense-v3` 环境中获得 >= -3.5 的结果。

要查找你的结果，请访问[排行榜](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard)并找到你的模型，**结果 = mean_reward - std_reward**

有关认证流程的更多信息，请查看此部分 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## 设置 GPU 💪
- 为了**加速 agent 的训练，我们将使用 GPU**。为此，请转到 `代码执行程序 > 更改运行时类型`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `硬件加速器 > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 创建虚拟显示器 🔽

在本 notebook 中，我们需要生成一个回放视频。为此，在 Colab 中，**我们需要一个虚拟屏幕来渲染环境**（从而记录帧）。

因此，下面的单元格将安装所需的库，并创建和运行一个虚拟屏幕 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# 虚拟显示器
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### 安装依赖项 🔽

第一步是安装依赖项，我们将安装多个库：
- `gymnasium`
- `panda-gym`：包含机械臂环境。
- `stable-baselines3`：SB3 深度强化学习库。
- `huggingface_sb3`：用于从 Hugging Face 🤗 Hub 加载和上传模型的 Stable-baselines3 附加代码。
- `huggingface_hub`：允许任何人使用 Hub 仓库的库。

⏲ 安装过程可能**需要 10 分钟**。

In [3]:
!pip install stable-baselines3[extra]
!pip install gymnasium

In [6]:
!pip install huggingface_sb3
!pip install huggingface_hub

# 在 Windows 上，pybullet 会编译失败。
# 先从https://visualstudio.microsoft.com/visual-cpp-build-tools/安装使用 C++ 的桌面开发编译工具
# 再从x64 Native Tools Command Prompt for VS 2022中激活conda,导航到项目目录再安装

!pip install pybullet
!pip install panda_gym

  Using cached pybullet-3.2.7.tar.gz (80.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pybullet
Failed to build pybullet


  DEPRECATION: Building 'pybullet' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pybullet'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [3577 lines of output]
      win-amd64
      numpy is enabled.
      numpy_include_dirs = D:\DevelopmentSoftware\Anaconda\envs\py311\Lib\site-packages\numpy\_core\include
      win32!
      found resource files: 3568
      -- block.urdf
      -- bunny.obj
      -- cartpole.urdf
      -- checker_blue.png
      -- checker_grid.jpg
      -- checker_huge.gif
      -- cloth_z_up.mtl
      -- cloth_z_up.obj
      -- clot

  Using cached panda_gym-3.0.7-py3-none-any.whl.metadata (4.3 kB)
  Using cached pybullet-3.2.7.tar.gz (80.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached panda_gym-3.0.7-py3-none-any.whl (23 kB)
  Running setup.py clean for pybullet
Failed to build pybullet


  DEPRECATION: Building 'pybullet' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pybullet'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [3577 lines of output]
      win-amd64
      numpy is enabled.
      numpy_include_dirs = D:\DevelopmentSoftware\Anaconda\envs\py311\Lib\site-packages\numpy\_core\include
      win32!
      found resource files: 3568
      -- block.urdf
      -- bunny.obj
      -- cartpole.urdf
      -- checker_blue.png
      -- checker_grid.jpg
      -- checker_huge.gif
      -- cloth_z_up.mtl
      -- cloth_z_up.obj
      -- clot

## 导入包 📦

In [7]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


## PandaReachDense-v3 🦾

我们要训练的 agent 是一个需要进行控制（移动手臂和使用末端执行器）的机械臂。

在机器人学中，*末端执行器*是位于机械臂末端、用于与环境交互的设备。

在 `PandaReach` 中，机器人必须将其末端执行器放置在目标位置（绿色球）。

我们将使用该环境的密集（dense）版本。这意味着我们将得到一个*密集奖励函数*，它**会在每个时间步提供奖励**（agent 越接近完成任务，奖励就越高）。这与*稀疏奖励函数*相反，后者**仅在任务完成时才返回奖励**。

此外，我们将使用*末端执行器位移控制*，这意味着**动作对应于末端执行器的位移**。我们不控制每个关节的单独运动（关节控制）。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/robotics.jpg"  alt="Robotics"/>


这样**训练会更容易**。



### 创建环境

#### 环境 🎮

在 `PandaReachDense-v3` 中，机械臂必须将其末端执行器放置在目标位置（绿色球）。

In [8]:
env_id = "PandaReachDense-v3"

# 创建环境
env = gym.make(env_id)

# 获取状态空间和动作空间
s_size = env.observation_space.shape
a_size = env.action_space

In [9]:
print("_____观测空间_____ \n")
print("状态空间为: ", s_size)
print("观测样本", env.observation_space.sample()) # 获取一个随机观测

_____观测空间_____ 

状态空间为:  None
观测样本 {'achieved_goal': array([-1.876294,  7.911396,  9.085486], dtype=float32), 'desired_goal': array([-7.6672406,  8.0997505, -6.360858 ], dtype=float32), 'observation': array([-4.559535  ,  5.18603   ,  9.865664  , -2.298166  , -7.51847   ,
        0.16748801], dtype=float32)}


观测空间是**一个包含 3 个不同元素的字典**：
- `achieved_goal`: (x,y,z) 末端执行器的当前位置。
- `desired_goal`: (x,y,z) 末端执行器的目标位置。
- `observation`: 末端执行器的位置 (x,y,z) 和速度 (vx, vy, vz)。

由于观测是一个字典，**我们将需要使用 MultiInputPolicy 策略而不是 MlpPolicy**。

In [10]:
print("\n _____动作空间_____ \n")
print("动作空间为: ", a_size)
print("动作空间样本", env.action_space.sample()) # 执行一个随机动作


 _____动作空间_____ 

动作空间为:  Box(-1.0, 1.0, (3,), float32)
动作空间样本 [0.44655457 0.39909253 0.32395583]


动作空间是一个包含 3 个值的向量：
- 控制 x, y, z 方向的移动

### 归一化观测和奖励

在强化学习中，一个好的做法是[归一化输入特征](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)。

为此，有一个包装器（wrapper）可以计算输入特征的移动平均值和标准差。

我们还通过添加 `norm_reward = True` 使用同一个包装器来归一化奖励。

[你应该查阅文档来填写此单元格](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)

---

## 🎯 核心参数建议
| 参数 | 推荐值 | 作用 |
|------|--------|------|
| `venv` | `env` | 需要包装的向量化环境 |
| `norm_obs` | `True` | 对观测值做归一化，稳定训练 |
| `norm_reward` | `True` | 对奖励做归一化，减少回报尺度差异的影响 |
| `clip_obs` | `10.0` | 限制观测值的绝对值范围，防止极端值干扰 |
| `clip_reward` | _可选，例如_ `10.0` | 限制奖励的绝对值（有些任务可以不设） |
| `gamma` | 与算法保持一致（如 `0.99`） | 折扣因子 |
| `epsilon` | `1e-8` | 避免除零错误 |
| `training` | `True` | 在训练时更新统计量，评估时应设为 `False` |
| `norm_obs_keys` | 默认为 `None` | 仅在观测是字典类型时需要指定键 |

---

## 💡 实用范例
```python
env = VecNormalize(
    env,
    norm_obs=True,
    norm_reward=True,
    clip_obs=10.0,
    clip_reward=10.0,
    gamma=0.99,
    epsilon=1e-8
)
```



In [ ]:
env = make_vec_env(env_id, n_envs=4)

# 添加此包装器以归一化观测和奖励
env = # TODO: 添加包装器

#### 参考答案

In [11]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10., gamma= 0.99, epsilon=1e-8)

### 创建 A2C 模型 🤖

有关使用 StableBaselines3 实现 A2C 的更多信息，请查看：https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

为了找到最佳参数，我查阅了 [Stable-Baselines3 团队官方训练的 agent](https://huggingface.co/sb3)。

In [ ]:
model = # 创建 A2C 模型并尝试找到最佳参数

#### 参考答案

In [12]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

Using cuda device


### 训练 A2C agent 🏃
- 让我们训练我们的 agent 1,000,000 个时间步，别忘了在 Colab 上使用 GPU。这大约需要 25-40 分钟。

In [13]:
model.learn(1_000_000)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 37.2     |
|    ep_rew_mean        | -8.47    |
|    success_rate       | 0.365    |
| time/                 |          |
|    fps                | 365      |
|    iterations         | 100      |
|    time_elapsed       | 5        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -4.29    |
|    explained_variance | -0.0583  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 2.18     |
|    std                | 1.01     |
|    value_loss         | 0.996    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 38.8     |
|    ep_rew_mean        | -8.96    |
|    success_rate       | 0.33     |
| time/                 |          |
|    fps                | 349      |
|    iterations         | 200      |
|

In [14]:
# 保存模型和 VecNormalize 的统计数据
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

### 评估 agent 📈
- 现在我们的 agent 已经训练好了，我们需要**检查它的性能**。
- Stable-Baselines3 提供了一个方法来做到这一点：`evaluate_policy`

In [15]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 加载已保存的统计数据
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# 我们需要覆盖 render_mode
eval_env.render_mode = "rgb_array"

# 在测试时不更新它们
eval_env.training = False
# 测试时不需要奖励归一化
eval_env.norm_reward = False

# 加载 agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"平均奖励 = {mean_reward:.2f} +/- {std_reward:.2f}")

平均奖励 = -0.19 +/- 0.11


D:\DevelopmentSoftware\Anaconda\envs\py311\Lib\site-packages\stable_baselines3\common\evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


### 将你训练好的模型发布到 Hub 🔥
既然我们看到训练后取得了不错的结果，我们就可以用一行代码将训练好的模型发布到 Hub 上。

📚 库文档 👉 https://github.com/huggingface/huggingface_sb3/tree/main#hugging-face--x-stable-baselines3-v20


通过使用 `package_to_hub`，正如我们在前面的单元中已经提到的，**你可以评估、记录回放、生成 agent 的模型卡并将其推送到 Hub**。

这样：
- 你可以**展示你的工作** 🔥
- 你可以**可视化你的 agent 的表现** 👀
- 你可以**与社区分享一个其他人可以使用的 agent** 💾
- 你可以**访问排行榜 🏆，查看你的 agent 与同学相比表现如何** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard


为了能与社区分享你的模型，还需要遵循以下三个步骤：

1️⃣ （如果尚未完成）创建一个 HF 账户 ➡ https://huggingface.co/join

2️⃣ 登录，然后你需要存储来自 Hugging Face 网站的身份验证令牌。
- 创建一个新令牌（https://huggingface.co/settings/tokens），**并赋予写入权限**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- 复制令牌
- 运行下面的单元格并粘贴令牌

In [16]:
notebook_login()
!git config --global credential.helper store

如果你不想使用 Google Colab 或 Jupyter Notebook，你需要改用这个命令：`huggingface-cli login`

3️⃣ 现在我们准备好使用 `package_to_hub()` 函数将我们训练好的 agent 推送到 🤗 Hub 🔥

对于这个环境，**运行此单元格可能需要大约 10 分钟**

In [18]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"a1024053774/a2c-{env_id}", # 更改用户名
    commit_message="初始提交",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to C:\Users\10240\AppData\Local\Temp\tmpil_57x1l\-step-0-to-step-1000.mp4
MoviePy - Building video C:\Users\10240\AppData\Local\Temp\tmpil_57x1l\-step-0-to-step-1000.mp4.
MoviePy - Writing video C:\Users\10240\AppData\Local\Temp\tmpil_57x1l\-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready C:\Users\10240\AppData\Local\Temp\tmpil_57x1l\-step-0-to-step-1000.mp4
✘ 'DummyVecEnv' object has no attribute 'video_recorder'
✘ We are unable to generate a replay of your agent, the package_to_hub
process continues
✘ Please open an issue at
https://github.com/huggingface/huggingface_sb3/issues
ℹ Pushing repo a1024053774/a2c-PandaReachDense-v3 to the Hugging Face
Hub


policy.pth:   0%|          | 0.00/46.8k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

a2c-PandaReachDense-v3.zip:   0%|          | 0.00/114k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/48.9k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.66k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/a1024053774/a2c-PandaReachDense-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/a1024053774/a2c-PandaReachDense-v3/commit/d37db533b6b18c600dced44c35ec00885124fa2c', commit_message='初始提交', commit_description='', oid='d37db533b6b18c600dced44c35ec00885124fa2c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/a1024053774/a2c-PandaReachDense-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='a1024053774/a2c-PandaReachDense-v3'), pr_revision=None, pr_num=None)

## 一些额外的挑战 🏆
最好的学习方式是**亲自动手尝试**！为什么不试试 `PandaPickAndPlace-v3`？

如果你想尝试更高级的 panda-gym 任务，你需要查看使用 **TQC 或 SAC**（一种更适合机器人任务的、样本效率更高的算法）完成的工作。在真实的机器人技术中，你会使用样本效率更高的算法，原因很简单：与模拟不同，**如果你过多地移动机械臂，就有损坏它的风险**。

PandaPickAndPlace-v1（此模型使用该环境的 v1 版本）：https://huggingface.co/sb3/tqc-PandaPickAndPlace-v1

不要犹豫，在此处查看 panda-gym 文档：https://panda-gym.readthedocs.io/en/latest/usage/train_with_sb3.html

我们为你提供训练另一个 agent 的步骤（可选）：

1. 定义名为 “PandaPickAndPlace-v3” 的环境
2. 创建一个向量化的环境
3. 添加一个包装器来归一化观测和奖励。[查阅文档](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)
4. 创建 A2C 模型（不要忘记设置 verbose=1 以打印训练日志）。
5. 训练 100 万个时间步
6. 保存模型和 VecNormalize 的统计数据
7. 评估你的 agent
8. 使用 `package_to_hub` 将你训练好的模型发布到 Hub 🔥


### 参考答案（可选）

In [19]:
# 1 - 2
env_id = "PandaPickAndPlace-v3"
env = make_vec_env(env_id, n_envs=4)

# 3
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 4
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)
# 5
model.learn(1_000_000)

Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 48.7     |
|    ep_rew_mean        | -48.6    |
|    success_rate       | 0.05     |
| time/                 |          |
|    fps                | 370      |
|    iterations         | 100      |
|    time_elapsed       | 5        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -5.69    |
|    explained_variance | 0.732    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.644   |
|    std                | 1        |
|    value_loss         | 0.0922   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 48.7     |
|    ep_rew_mean        | -48.7    |
|    success_rate       | 0.0375   |
| time/                 |          |
|    fps                | 346      |
|    iterations     

In [20]:
# 6
model_name = "a2c-PandaPickAndPlace-v3";
model.save(model_name)
env.save("vec_normalize.pkl")

# 7
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 加载已保存的统计数据
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# 在测试时不更新它们
eval_env.training = False
# 测试时不需要奖励归一化
eval_env.norm_reward = False

# 加载 agent
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"平均奖励 = {mean_reward:.2f} +/- {std_reward:.2f}")

# 8
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"a1024053774/a2c-{env_id}", # TODO: 更改用户名
    commit_message="初始提交",
)

平均奖励 = -50.00 +/- 0.00
ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to C:\Users\10240\AppData\Local\Temp\tmpc59vrnmr\-step-0-to-step-1000.mp4
MoviePy - Building video C:\Users\10240\AppData\Local\Temp\tmpc59vrnmr\-step-0-to-step-1000.mp4.
MoviePy - Writing video C:\Users\10240\AppData\Local\Temp\tmpc59vrnmr\-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready C:\Users\10240\AppData\Local\Temp\tmpc59vrnmr\-step-0-to-step-1000.mp4
✘ 'DummyVecEnv' object has no attribute 'video_recorder'
✘ We are unable to generate a replay of your agent, the package_to_hub
process continues
✘ Please open an issue at
https://github.com/huggingface/huggingface_sb3/issues
ℹ Pushing repo a1024053774/a2c-PandaPickAndPlace-v3 to the Hugging Face
Hub


pytorch_variables.pth:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/53.7k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

a2c-PandaPickAndPlace-v3.zip:   0%|          | 0.00/130k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/a1024053774/a2c-PandaPickAndPlace-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/a1024053774/a2c-PandaPickAndPlace-v3/commit/6073e1630aaddc62f47d658e02c094179e1d0f9c', commit_message='初始提交', commit_description='', oid='6073e1630aaddc62f47d658e02c094179e1d0f9c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/a1024053774/a2c-PandaPickAndPlace-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='a1024053774/a2c-PandaPickAndPlace-v3'), pr_revision=None, pr_num=None)

第 7 单元见！🔥
## 保持学习，保持热情 🤗